# Setup
## Imports

In [ ]:
# Standard Python libraries
import sys
import random
import string
import os
import math

from ultralytics import YOLO

# Third party libraries
from pylibdmtx.pylibdmtx import encode
from PIL import Image, ImageFont, ImageDraw
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Custom funcs
sys.path.append('../scripts')
from hourglass import *

## YOLO setup

In [ ]:
YOLO_model = YOLO(f'../yolo/runs/obb/train8/weights/best.pt') # load best model from training
YOLO_model.eval() # Set the model to evaluation mode

## Hourglass setup

In [ ]:
input_size = 256 # Input image size for model
output_size = 64 # Output heatmap size for model
n_stacks = 8     # Number of stacks in hourglass model

sigma = 1        # Gaussian distribution (spread) for heatmap generation

n_train = 800    # Number of training samples
n_val = 100      # Number of validation samples
n_test = 100     # Number of test samples

batch_size = 1   # Batch size for training

max_complex_epoch = 150 # Number of epochs where complex data is used

# Calculating weight for loss function
# pixels_per_gauss = (5 + 7 + (9*5) + 7 + 5) * 4 # Circles are built from vertical pixels of 5, 7, 9, 9, 9, 9, 9, 7, 5 (and there are 4 of them)
pixels_per_gauss = (3 + 5 + 5 + 5 + 3) * 4 # Circles are built from vertical pixels of 3, 5, 5, 5, 3
pixels_total = output_size*output_size
weight = (pixels_total - pixels_per_gauss) / pixels_per_gauss
print(weight) # There are this many times more background pixels than gaussian pixels

reset_synth = True # Set to True to regenerate synthetic data

# Hourglass Training